In [2]:

import numpy as np
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Lambda
from keras.utils import np_utils
import h5py

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Processing

In [7]:
import data_processing
import PN_tools

In [8]:
 train_points_r, test_points_r, Y_train, Y_test = data_processing.load_data()

/datasets/home/home-01/25/125/a3mai/Pointnet/Prepdata
['ply_data_train4.h5', '.ipynb_checkpoints', 'ply_data_train2.h5', 'ply_data_train3.h5', 'ply_data_train1.h5', 'ply_data_train0.h5']
/datasets/home/home-01/25/125/a3mai/Pointnet/Prepdata_test
['.ipynb_checkpoints', 'ply_data_test1.h5', 'ply_data_test0.h5']


In [9]:
train_points_r.shape

(9840, 2048, 3)

# Model

In [10]:
import input_Tnet
import feature_Tnet

In [11]:

# number of points in each sample
num_points = train_points_r.shape[1]

# number of categories
k = 40

#  Pointnet Architecture
adam = optimizers.Adam(lr=0.001, decay=0.7)
input_points = Input(shape=(num_points, 3))

# T_net(transformation_net)
input_T = input_Tnet.Input_Tnet(num_points, input_points)

# Foward_net
fw = feature_Tnet.forward_net(input_T, input_points, num_points)

# T_net(transformation_net)
feature_T = feature_Tnet.feature_Tnet(fw, num_points)

# Forward net_v2
fw_v2 = feature_Tnet.forward_net_v2(feature_T, fw)

# Golobal_features
global_feature = MaxPooling1D(pool_size=num_points)(fw_v2)

# Convert pointnet to make prediction
a = Dense(512, activation='relu')(global_feature)
a = BatchNormalization()(a)
a = Dropout(rate=0.7)(a)
a = Dense(256, activation='relu')(a)
a = BatchNormalization()(a)
a = Dropout(rate=0.7)(a)
a = Dense(k, activation='softmax')(a)
prediction = Flatten()(a)


PN_model = Model(inputs=input_points, outputs=prediction)
print(PN_model.summary())



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2048, 3)           0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 2048, 3)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2048, 64)          256       
_________________________________________________________________
batch_normalization_6 (Batch (None, 2048, 64)          256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 2048, 64)          4160      
_________________________________________________________________
batch_normalization_7 (Batch (None, 2048, 64)          256       
_________________________________________________________________
lambda_2 (Lambda)            (None, 2048, 64)          0         
__________

# Training

In [ ]:

PN_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
for i in range(1,50):
    train_points_rotate = PN_tools.rotate_point_cloud(train_points_r)
    train_points_jitter = PN_tools.jitter_point_cloud(train_points_rotate)
    history = PN_model.fit(train_points_jitter, Y_train, batch_size=32, epochs=1, shuffle=True, verbose=1)
    s = "Current epoch is:" + str(i)
    print(s)
# score the model
score = PN_model.evaluate(test_points_r, Y_test, verbose=1)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])


Epoch 1/1
9840/9840 [==============================] - 81s 8ms/step - loss: 2.1072 - acc: 0.4284
Current epoch is:1
Epoch 1/1
1824/9840 [====>.........................] - ETA: 1:02 - loss: 1.8041 - acc: 0.4874

In [84]:
t = PN_model.predict(test_points_r)

In [85]:
np.save('predict_pn1.npy', t)

In [86]:
np.save('test_point1.npy', test_points_r)

In [87]:
np.save('test_label1.npy', Y_test)